In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
df = pd.read_csv("labeled_thermal_features.csv")

# Use only normal data (Porosity Label = 0) for training GAN
df_normal = df[df["Porosity Label"] == 0].drop(columns=["Frame", "Porosity Label"])
df_anomalies = df[df["Porosity Label"] == 1].drop(columns=["Frame", "Porosity Label"])

# Normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(df_normal)
X_test = scaler.transform(df.drop(columns=["Frame", "Porosity Label"]))

# Define Generator
def build_generator(latent_dim, output_dim):
    model = keras.Sequential([
        layers.Dense(32, activation="relu", input_dim=latent_dim),
        layers.Dense(64, activation="relu"),
        layers.Dense(output_dim, activation="sigmoid")  # Sigmoid for normalized data
    ])
    return model

# Define Discriminator
def build_discriminator(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_dim=input_dim),
        layers.Dense(32, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification
    ])
    return model

# Hyperparameters
latent_dim = 10  # Size of random noise
data_dim = X_train.shape[1]
epochs = 5000
batch_size = 32

# Build GAN components
generator = build_generator(latent_dim, data_dim)
discriminator = build_discriminator(data_dim)

# Compile Discriminator
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0002), metrics=["accuracy"])

# Build and compile GAN
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(0.0002))

# Training GAN
for epoch in range(epochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_samples = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_samples = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_samples, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_samples, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    misleading_labels = np.ones((batch_size, 1))  # Fool the discriminator
    g_loss = gan.train_on_batch(noise, misleading_labels)

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: D Loss: {d_loss[0]:.4f}, G Loss: {g_loss:.4f}")

print("✅ GAN Training Completed!")

# **Use Discriminator for Anomaly Detection**
discriminator.trainable = True
anomaly_scores = discriminator.predict(X_test)

# Set a threshold for anomaly detection (e.g., bottom 5% confidence)
threshold = np.percentile(anomaly_scores, 5)
anomaly_predictions = (anomaly_scores < threshold).astype(int)

# Evaluate model
from sklearn.metrics import confusion_matrix, classification_report
y_true = df["Porosity Label"].values

print("\nGAN-Based Anomaly Detection Evaluation:")
print("Confusion Matrix:\n", confusion_matrix(y_true, anomaly_predictions))
print("Classification Report:\n", classification_report(y_true, anomaly_predictions))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


/Users/berkeayyildizli/ENS_Graduation_Project/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/berkeayyildizli/ENS_Graduation_Project/.venv/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: D Loss: 0.7179, G Loss: 0.7661
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━